<img src="../images/Lektion5.png" style="margin: 20px 0px 20px 0px"/>
<h2 style="display:none">Lektion 5 - Vererbung</h2>


In Kotlin kann nicht von einer beliebigen Klasse geerbt werden. Diese muss als Voraussetzung den Vorsatz `open` besitzen. Dadurch wird signalisiert, dass eine Vererbung stattfinden darf. Der Namen der Klasse, von der geerbt werden soll, ist mit einem `:` getrennt hinter den Parameterklammern der erbenden Klasse zu schreiben. Dem Konstruktor der Super-Klasse werden die Parameter direkt im Kopf der Klasse übergeben. 

In [ ]:
open class Person(val name: String, var alter: Int) { //Super-Klasse
    override fun toString() = "Mein Name ist $name und ich bin $alter Jahre alt."
}

class Student(name: String, alter: Int) : Person(name, alter){ //Sub-Klasse, die von Person erbt
    companion object {
        var zaehler = 0
            get() = field++
    }
    val matrikelnummer = zaehler
    
    override fun toString() = "Mein Name ist $name, ich bin $alter Jahre alt und besitze die Matrikelnummer $matrikelnummer."
}

val p1 = Person ("Max Mustermann", 25)
val s1 = Student ("Erika Mustermann", 21)
println("p1: $p1")
println("s1: $s1")

### Felder und Methoden überschreiben
Sollen Felder oder Methoden der Super-Klasse in der Sub-Klasse überschrieben werden, müssen diese ebenfalls mit `open` gekennzeichnet werden. Erst dann ist es möglich das gleiche Feld oder die gleiche Methode in der Sub-Klasse mit dem Zusatz `override` zu definieren. Soll auf das Feld oder die Methode der Oberklasse zugegriffen werden, findet das Schlüsselwort `super` Anwendung. Bei Mehrfachvererbung wird die nächst höhere Klasse angesprochen.

In [ ]:
open class Person(val name: String, var alter: Int){
    open val schlachtruf //Dieses Feld kann von den Sub-Klassen überschrieben werden
        get() = "Für die Horde!"
    
    open fun hatGeburtstag() { //Diese Methode kann von den Sub-Klassen überschrieben werden
        println("$name hat Geburtstag. Alles Gute!")
        alter++
    } 
    override fun toString() = "Mein Name ist $name und ich bin $alter Jahre alt. $schlachtruf"
}

class Student(name: String, alter: Int) : Person(name, alter){
    companion object {
        var zaehler = 0
            get() = field++
    }
    val matrikelnummer = zaehler
    
    override val schlachtruf
        get() = "Für die Allianz!"
    
    override fun hatGeburtstag() { //Überschreibt die Methode hatGeburtstag der Super-Klasse
        println("$name hat Geburtstag. Alles Gute!")
        alter--
    } 
    
    override fun toString() = "Mein Name ist $name, ich bin $alter Jahre alt und habe die Matrikelnummer $matrikelnummer. $schlachtruf Oder vielleicht doch: ${super.schlachtruf}?"
}

val p1 = Person ("Max Mustermann", 25)
val s1 = Student ("Erika Mustermann", 21)
println("p1: $p1")
p1.hatGeburtstag() //Ruft hatGeburtstag() von der Klasse Person auf 
println("p1: $p1")
println("s1: $s1")
s1.hatGeburtstag() //Ruft hatGeburtstag() von der Klasse Student auf 
println("s1: $s1")

### Sekundäre Konstruktoren
Wir sind bis jetzt von dem einfachen Fall ausgegangen, dass die Unterklasse einen primären Konstruktor besitzt. Jedoch gibt es auch die Möglichkeit diese nur mit sekundären Konstruktoren auszustatten. Diese müssen immer den primären Konstruktor der Oberklasse aufrufen.

In [ ]:
open class Punkt2D (val x: Int, val y: Int){
    override fun toString() = "x: $x, y: $y"
}

class Punkt3D : Punkt2D { //Aufruf des primären Konstruktors von Punkt2D fehlt hier
    val z: Int
    
    //sekundäre Konstruktoren rufen den primären Defaultkonstruktor von Punkt2D auf
    constructor (x: Int, y: Int, z: Int) : super (x, y) {
        this.z = z
    }
    constructor () : super (0, 0) {
        this.z = 0
    }
    
    override fun toString() = "${super.toString()}, z: $z"
}

val p1 = Punkt3D (4, 2, 8)
println("Punkt p1: $p1")
val p2 = Punkt3D()
println("Punkt p2: $p2")

### Abstrakte Klassen
Das Konzept der abstrakten Klasse sollte bereits aus Java bekannt sein. Auch in Kotlin gibt es die Möglichkeit uninitierbare Klassen mit abstrakten Methoden zu implementieren. Diesen wird das Schlüsselwort `abstract` vorangestellt und können ohne Probleme vererbt werden. Die Sub-Klassen müssen jedoch die Methoden vervollständigen.

In [ ]:
abstract class Computer (val kerne: Int, val farbe: String){ //Abstrakte Klasse
    var istAn = false
    abstract fun einschalten() //Abstrakte Methode
}

class Laptop (kerne: Int, farbe: String, val hatTouchscreen: Boolean) : Computer(kerne, farbe){
    var istAufgeklappt = false
    override fun einschalten() { //Vervollständigt die abstrakte Methode einschalten(), die geerbt wurde
        if (!istAufgeklappt) 
            istAufgeklappt = true
        istAn = true
        println("Laptop wurde eingeschalten.")
    }
}

class Standcomputer (kerne: Int, farbe: String) : Computer(kerne, farbe){
    override fun einschalten() {
        istAn = true
        println("Standcomputer wurde eingeschalten.")
    }
}

//val c1 = Computer (8, "Schwarz") -> Error: Cannot create an instance of an abstract class
val l1 = Laptop(6, "Rot", false)
l1.einschalten()
val s1 = Standcomputer(12, "Weiß")
s1.einschalten()

### Geschlossene Klassen
Eine geschlossene Klasse ist eine besondere abstrakte Klasse. Ihr wird das Schlüsselwort `sealed` vorangestellt. Sie besitzt einen privaten Konstruktor. Von dieser darf nur in dem selben Paket oder der selben Kompiliereinheit geerbet werden. Dieses Konstrukt ist somit eine Alternative zu einer Enumeration. Da alle Subtypen bekannt sind, kann bei Verwendung einer `when`-Verzweigung der *default*-Fall weggelassen werden.

In [ ]:
sealed class Fehler
data class Laufzeitfehler(val fehlercode: Int) : Fehler()
data class Kompilierfehler(val fehlercode: Int) : Fehler()
data class Referenzfehler(val fehlercode: Int) : Fehler()

val zufall = (0..2).random()
val f1 = if(zufall == 1)
                Laufzeitfehler(123)
            else if(zufall == 2)
                Kompilierfehler(42)
            else
                Referenzfehler(200)

when (f1){
    is Laufzeitfehler -> println("Es ist ein Laufzeitfehler $f1 aufgetreten.")
    is Kompilierfehler -> println("Es ist ein Kompilierfehler $f1 aufgetreten.")
    is Referenzfehler -> println("Es ist ein Referenzfehler $f1 aufgetreten.")
}